# Background Information on Datasets

# 1. Load Required Libraries

In [57]:
# Load Required Libraries
import pandas as pd; 
import numpy as np; 
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.figure import Figure
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
import param
import hvplot.pandas
import panel as pn
pn.extension()
mpl.use('agg')




from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report

# 2. Read Data Files

In [58]:
# Read Data Files
dataset1_csv_file = "data/Dataset2.csv"
df1 = pd.read_csv(dataset1_csv_file)

dataset2_csv_file = "data/Credit Card Customer Churn.csv"
df2 = pd.read_csv(dataset2_csv_file)

# 3. Understanding the Datasets

In [59]:
# peek into dataset 1
df1.head()
df1.tail()

# peek into dataset 2
df2.head()
df2.tail()

,CUST_ID,SEX,AGE,EDUCATION,INVESTMENT,INCOME,ACTIVITY,CHURN,YRLY_AMT,AVG_DAILY_TX,YRLY_TX,AVG_TX_AMT,NEGTWEETS,STATE,EDUCATION_GROUP,TWITTERID,CHURN_LABEL
5996,1009569950,M,66,1,0,13024,1,0,6638.5510,0.660274,241,27.545898,4,NJ,High school graduate,0.0,False
5997,1009541780,M,29,1,0,13020,4,0,6094.6406,0.657534,240,25.394302,4,NV,High school graduate,0.0,False
5998,1009550850,M,33,1,0,13010,1,1,6348.0900,0.668493,244,26.016800,14,ND,High school graduate,0.0,True
5999,1009527790,M,61,4,0,13007,3,0,7482.5117,0.690411,252,29.692505,3,CA,Associate degree,0.0,False
6000,1009572520,F,58,1,0,13003,3,0,6453.8710,0.665753,243,26.559097,1,AK,High school graduate,0.0,False


Since we have two different datasets, we need to perform data standardization and cleaning before we proceed with our analysis. As we can see, in the second dataset, "Customer"

In [60]:
# Format all columns
# print original data frame
print(df2.columns)

# remove special character
df2.columns = df2.columns.str.replace(' ', '')

# Remname the columns in df2 to match the column in df1
df2 = df2.rename(columns={'CUST_ID': 'CustomerId', 'AGE':'Age', 'INCOME':'EstimatedSalary', 'CHURN':'Exited', 
                          'INVESTMENT': 'Investment', 'ACTIVITY':'Activity', 'YRLY_AMT':'Yearly Amt', 
                          'AVG_DAILY_TX':'Avg Daily Tax', 'AVG_DAILY_TX':'Avg Daily Tax',
                          'YRLY_TX':'Yearly Tax', 'AVG_TX_AMT':'Avg Tax Amt', 'NEGTWEETS':'Negtweets', 'STATE':'State',
                          'EDUCATION':'Education',
                          'EDUCATION_GROUP':'Education Group', 'TWITTERID':'TwitterID'
                          })

# Since dataset is for all the different states in US, we can assume that the Geography value as 'United States'
if 'Geography' not in df2:
    df2.insert(loc=1,
            column='Geography',
            value='United States')

# Check how many unique values in 'SEX' column df2
print("Unique Values for Sex Column: ", pd.unique(df2['SEX']))

# Applying the condition to create a gender column based on the value in SEX column of the df2
df2['Gender'] = np.where((df2['SEX']  == 'F'), 'Female', 'Male')

df2.describe()

Index(['CUST_ID', 'SEX', 'AGE', 'EDUCATION', 'INVESTMENT', 'INCOME',
       'ACTIVITY', 'CHURN', ' YRLY_AMT', 'AVG_DAILY_TX', 'YRLY_TX',
       'AVG_TX_AMT', 'NEGTWEETS', 'STATE', 'EDUCATION_GROUP', 'TWITTERID',
       ' CHURN_LABEL'],
      dtype='object')
Unique Values for Sex Column:  ['F' 'M']


,CustomerId,Age,Education,Investment,EstimatedSalary,Activity,Exited,Yearly Amt,Avg Daily Tax,Yearly Tax,Avg Tax Amt,Negtweets,TwitterID
count,6.001000e+03,6001.000000,6001.000000,6001.000000,6.001000e+03,6001.000000,6001.000000,6001.000000,6001.000000,6001.000000,6001.000000,6001.000000,6.001000e+03
mean,1.009550e+09,48.504083,2.100650,8788.980503,3.689169e+04,2.199133,0.254791,12527.073380,0.683584,249.508249,48.280455,4.805699,2.749704e+14
std,1.732484e+04,16.062950,1.367431,25914.903560,1.184143e+05,1.415217,0.435780,21869.459623,0.039624,14.462767,64.069223,3.534436,1.506082e+16
min,1.009520e+09,20.000000,1.000000,0.000000,1.300300e+04,0.000000,0.000000,6094.640600,0.649315,237.000000,25.394302,0.000000,0.000000e+00
25%,1.009535e+09,36.000000,1.000000,0.000000,1.687300e+04,1.000000,0.000000,8053.390600,0.660274,241.000000,32.864304,3.000000,0.000000e+00
50%,1.009550e+09,47.000000,1.000000,0.000000,2.052400e+04,2.000000,0.000000,9243.262000,0.665753,243.000000,37.658005,3.000000,0.000000e+00
75%,1.009565e+09,60.000000,4.000000,0.000000,2.957500e+04,3.000000,1.000000,12161.602000,0.698630,255.000000,49.571396,5.000000,0.000000e+00
max,1.009580e+09,90.000000,5.000000,119927.000000,3.852862e+06,5.000000,1.000000,726977.000000,0.994521,363.000000,2095.040000,19.000000,8.275482e+17


# 4. Merge the two data files

In [61]:
merged_df = pd.merge(df1, df2, on=['CustomerId', 'Age', 'Gender', 'EstimatedSalary', 'Exited', 'Geography'], how='outer')


#check raw data
print(merged_df.head())

merged_df

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0        1.0    15634602  Hargrave        619.0    France  Female   42   
1        2.0    15647311      Hill        608.0     Spain  Female   41   
2        3.0    15619304      Onio        502.0    France  Female   42   
3        4.0    15701354      Boni        699.0    France  Female   39   
4        5.0    15737888  Mitchell        850.0     Spain  Female   43   

   Tenure     Balance  NumOfProducts  ...  Activity  Yearly Amt  \
0     2.0         0.0            1.0  ...       NaN         NaN   
1     1.0   8380786.0            1.0  ...       NaN         NaN   
2     8.0   1596608.0            3.0  ...       NaN         NaN   
3     1.0         0.0            2.0  ...       NaN         NaN   
4     2.0  12551082.0            1.0  ...       NaN         NaN   

   Avg Daily Tax  Yearly Tax Avg Tax Amt  Negtweets  State  Education Group  \
0            NaN         NaN         NaN        NaN    NaN              N

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,...,Activity,Yearly Amt,Avg Daily Tax,Yearly Tax,Avg Tax Amt,Negtweets,State,Education Group,TwitterID,CHURN_LABEL
0,1.0,15634602,Hargrave,619.0,France,Female,42,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,15647311,Hill,608.0,Spain,Female,41,1.0,8380786.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,15619304,Onio,502.0,France,Female,42,8.0,1596608.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,15701354,Boni,699.0,France,Female,39,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,15737888,Mitchell,850.0,Spain,Female,43,2.0,12551082.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15996,NaN,1009569950,NaN,NaN,United States,Male,66,NaN,NaN,NaN,...,1.0,6638.5510,0.660274,241.0,27.545898,4.0,NJ,High school graduate,0.0,False
15997,NaN,1009541780,NaN,NaN,United States,Male,29,NaN,NaN,NaN,...,4.0,6094.6406,0.657534,240.0,25.394302,4.0,NV,High school graduate,0.0,False
15998,NaN,1009550850,NaN,NaN,United States,Male,33,NaN,NaN,NaN,...,1.0,6348.0900,0.668493,244.0,26.016800,14.0,ND,High school graduate,0.0,True
15999,NaN,1009527790,NaN,NaN,United States,Male,61,NaN,NaN,NaN,...,3.0,7482.5117,0.690411,252.0,29.692505,3.0,CA,Associate degree,0.0,False


# Peek into the final dataset

In [62]:
# peek into dataset
merged_df.head()
merged_df.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,...,Activity,Yearly Amt,Avg Daily Tax,Yearly Tax,Avg Tax Amt,Negtweets,State,Education Group,TwitterID,CHURN_LABEL
15996,NaN,1009569950,NaN,NaN,United States,Male,66,NaN,NaN,NaN,...,1.0,6638.5510,0.660274,241.0,27.545898,4.0,NJ,High school graduate,0.0,False
15997,NaN,1009541780,NaN,NaN,United States,Male,29,NaN,NaN,NaN,...,4.0,6094.6406,0.657534,240.0,25.394302,4.0,NV,High school graduate,0.0,False
15998,NaN,1009550850,NaN,NaN,United States,Male,33,NaN,NaN,NaN,...,1.0,6348.0900,0.668493,244.0,26.016800,14.0,ND,High school graduate,0.0,True
15999,NaN,1009527790,NaN,NaN,United States,Male,61,NaN,NaN,NaN,...,3.0,7482.5117,0.690411,252.0,29.692505,3.0,CA,Associate degree,0.0,False
16000,NaN,1009572520,NaN,NaN,United States,Female,58,NaN,NaN,NaN,...,3.0,6453.8710,0.665753,243.0,26.559097,1.0,AK,High school graduate,0.0,False


In [63]:
# See all the columns in the dataset
merged_df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'SEX', 'Education',
       'Investment', 'Activity', 'Yearly Amt', 'Avg Daily Tax', 'Yearly Tax',
       'Avg Tax Amt', 'Negtweets', 'State', 'Education Group', 'TwitterID',
       'CHURN_LABEL'],
      dtype='object')

In [64]:
# General Information of dataset
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16001 entries, 0 to 16000
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  float64
 1   CustomerId       16001 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  float64
 4   Geography        16001 non-null  object 
 5   Gender           16001 non-null  object 
 6   Age              16001 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  float64
 10  HasCrCard        10000 non-null  float64
 11  IsActiveMember   10000 non-null  float64
 12  EstimatedSalary  16001 non-null  int64  
 13  Exited           16001 non-null  int64  
 14  SEX              6001 non-null   object 
 15  Education        6001 non-null   float64
 16  Investment       6001 non-null   float64
 17  Activity    

# Remove Unnecessary Columns from the merged datasets

In [65]:
merged_df.drop(['RowNumber','CustomerId','Surname', 'SEX', 'CHURN_LABEL', 'State', 'Negtweets','TwitterID'],axis=1,inplace=True)

merged_df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited', 'Education', 'Investment', 'Activity', 'Yearly Amt',
       'Avg Daily Tax', 'Yearly Tax', 'Avg Tax Amt', 'Education Group'],
      dtype='object')

# Dropping Missing Values

In [66]:
drop_row_all = merged_df.dropna(how = 'all')
missings = drop_row_all.isna().sum().sum()
print(f'Missings:{missings}')

Missings:116006


# Checking for missing values

In [67]:
# Check Missing Values in columns
merged_df.isna().sum()

# merged_df.to_csv('data/merged.csv', index=False)

CreditScore         6001
Geography              0
Gender                 0
Age                    0
Tenure              6001
Balance             6001
NumOfProducts       6001
HasCrCard           6001
IsActiveMember      6001
EstimatedSalary        0
Exited                 0
Education          10000
Investment         10000
Activity           10000
Yearly Amt         10000
Avg Daily Tax      10000
Yearly Tax         10000
Avg Tax Amt        10000
Education Group    10000
dtype: int64

# Replacing missing values

In [68]:
# Filling the missing data with the mean or median value if it’s a numerical variable.
# Filling the missing data with mode if it’s a categorical value.
# For state, which is only applicable for US and not applicable for other countries, we can consider to fill it with NA = Not Applicable

merged_df['CreditScore']=merged_df['CreditScore'].fillna(merged_df['CreditScore'].mean())
merged_df['Balance']=merged_df['Balance'].fillna(merged_df['Balance'].mean())
merged_df['NumOfProducts']=merged_df['NumOfProducts'].fillna(merged_df['NumOfProducts'].mean())


merged_df['Investment']=merged_df['Investment'].fillna(merged_df['Investment'].mean())
merged_df['Activity']=merged_df['Activity'].fillna(merged_df['Activity'].mean())
merged_df['Yearly Amt']=merged_df['Yearly Amt'].fillna(merged_df['Yearly Amt'].mean())
merged_df['Avg Daily Tax']=merged_df['Avg Daily Tax'].fillna(merged_df['Avg Daily Tax'].mean())
merged_df['Yearly Tax']=merged_df['Yearly Tax'].fillna(merged_df['Yearly Tax'].mean())
merged_df['Avg Tax Amt']=merged_df['Avg Tax Amt'].fillna(merged_df['Avg Tax Amt'].mean())
merged_df['Education']=merged_df['Education'].fillna(merged_df['Education'].mean())

merged_df['Education Group']=merged_df['Education Group'].fillna(merged_df['Education Group'].mode()[0])

merged_df['Tenure']=merged_df['Tenure'].fillna(merged_df['Tenure'].mean())

from sklearn import impute
im = impute.SimpleImputer(missing_values=np.nan, strategy='median')

merged_df['HasCrCard'] = im.fit_transform(merged_df[['HasCrCard']])
merged_df['IsActiveMember'] = im.fit_transform(merged_df[['IsActiveMember']])
# merged_df.info()
merged_df.isna().sum()
print("Unique Values for HasCrCard: ", pd.unique(merged_df['HasCrCard']))
print("Unique Values for IsActiveMember: ", pd.unique(merged_df['IsActiveMember']))

merged_df.to_csv('data/merged.csv', index=False)

Unique Values for HasCrCard:  [1. 0.]
Unique Values for IsActiveMember:  [1. 0.]


## Preliminary Data Analysis

**How many of the customer has credit card?**

In [69]:
num_customers_with_cc = len(merged_df[merged_df['HasCrCard'] == 1]['Age'].unique())

print(f"Number of customers with credit cards: {num_customers_with_cc}")

Number of customers with credit cards: 74


***<p style="color:blue">Number of customers with credit cards: 68. This indicates that only a small fraction of customers have credit cards with the bank.</p>***


**How many are active member?**

In [70]:
num_active_members = merged_df['IsActiveMember'].sum()

print('Number of active members:', num_active_members)

Number of active members: 11152.0


***<p style="color:blue">
Number of active members: 8242. This indicates that the bank has a significant number of active members who are actively using their services.</p>***

**How many customers have existing the bank?**

In [71]:
num_existing_members = (merged_df['Exited'] == 0).sum()

print('Number of existing members:', num_existing_members)

Number of existing members: 12435


***<p style="color:blue">Number of existing members: 12435. This indicates that the majority of customers are still with the bank and have not exited.</p>***


**What are the gender of customers?**

In [72]:
gender_count = merged_df['Gender'].value_counts()
print(gender_count)


Male      8536
Female    7465
Name: Gender, dtype: int64


*****<p style="color:blue">Gender count: Male - 8536, Female - 7465. This indicates that there is no significant gender imbalance in the dataset.</p>*****


**Plot the age distribution using a histogram**

In [73]:
plt.hist(merged_df['Age'])

# Add labels and title
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Age Distribution')

# Show plot
plt.show()

***<p style="color:blue">Age distribution: The age distribution is roughly normally distributed with a peak around 40 years. This indicates that the majority of customers are middle-aged.</p>***


**Propotion of different income level**


In [74]:
corr_matrix = merged_df[['Investment', 'EstimatedSalary']].corr()

# print the correlation matrix
print(corr_matrix)


                 Investment  EstimatedSalary
Investment         1.000000         0.004364
EstimatedSalary    0.004364         1.000000


***<p style="color:blue">Correlation matrix: The correlation between Investment and EstimatedSalary is very weak (correlation coefficient = 0.004364). This indicates that there is no significant correlation between these two features.</p>***


**Distribution of Total Number of Products**

In [75]:
# Plot histogram
plt.hist(merged_df['NumOfProducts'], bins=10)

# Add title and axis labels
plt.title('Distribution of NumOfProducts')
plt.xlabel('NumOfProducts')
plt.ylabel('Frequency')

# Show plot
plt.show()

***<p style="color:blue">Distribution of NumOfProducts: The distribution is right-skewed and the majority of customers have only 1 or 2 products with the bank.</p>***


**Distribution of Credit Limit**

In [76]:
# Create a histogram of CreditScore
plt.hist(merged_df['CreditScore'], bins=20, edgecolor='black')

# Set the title and axis labels
plt.title('Distribution of CreditScore')
plt.xlabel('CreditScore')
plt.ylabel('Count')

# Show the plot
plt.show()

***<p style="color:blue">Distribution of CreditScore: The distribution is roughly normal and the majority of customers have a credit score between 600 and 800.</p>***


****Proportion of exited vs not exited customers****

In [77]:
exited_count = merged_df['Exited'].value_counts()
total_count = exited_count.sum()

# Calculate the proportion of exited and non-exited customers
exited_percent = exited_count[1] / total_count * 100
not_exited_percent = exited_count[0] / total_count * 100

# Print the results
print(f"Percentage of Exited Customers: {exited_percent:.2f}%")
print(f"Percentage of Not-Exited Customers: {not_exited_percent:.2f}%")

Percentage of Exited Customers: 22.29%
Percentage of Not-Exited Customers: 77.71%


***<p style="color:blue">Percentage of Exited Customers: 22.29%. This indicates that the bank has a significant number of customers who have exited, which is a cause of concern. The bank should investigate the reasons for customer churn and take appropriate actions to retain customers.</p>***